# Hermitcraft Season 9 Stats

This notebook summarize minecraft stats files across all worlds saved on the local machine (currently OS X).  I would love to see this ported to work on Windows files, but additional work is required.

The emphasis is on creating plots that span worlds and compare worlds. Increasingly the focus is on statistics specific to hardcore mode.

Successfully executing the code will create a html file:  MinecraftStats.html.  The local_user variable needs to be modified to work your system. Comments in the notebook itself address the data analysis process.


In [97]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import gzip
import hvplot.pandas
from bokeh.resources import INLINE
import bokeh
import panel as pn
import holoviews as hv
import os

pn.extension(sizing_mode="stretch_width")
hv.extension("bokeh")

hermits = [
    {
        "profile": "Docm77",
        "hermit_key": "05e88dce-714d-4218-be77-fade8b5dfa3c",
    },
    {
        "profile": "isGall85",
        "hermit_key": "18a9faba-1977-469b-8156-ed0f1a3c765b",
    },
    {
        "profile": "Xisuma",
        "hermit_key": "21ef397c-3a76-4eb7-aa17-a99d3fc658e2",
    },
    {
        "profile": "Renthedog",
        "hermit_key": "2dd0cc3b-0825-4c3e-bd99-3bf07ef27447",
    },
    {
        "profile": "VintageBeef",
        "hermit_key": "2f723150-24de-44ff-aeee-87c75f7c7a9e",
    },
    {
        "profile": "iJevin",
        "hermit_key": "3f28c559-0898-4be1-9f20-9fd37ca9cd22",
    },
    {
        "profile": "joehillssays",
        "hermit_key": "53bae456-dbbb-4c2f-8c79-9e8ec26c8382",
    },
    {
        "profile": "GeminiTay",
        "hermit_key": "5a1839d2-cecc-4c85-aa08-b346f9f772a1",
    },
    {
        "profile": "Grian",
        "hermit_key": "5f8eb73b-25be-4c5a-a50f-d27d65e30ca0",
    },
    {
        "profile": "Tango",
        "hermit_key": "62fec5a3-1896-4beb-94e0-36e34898c787",
    },
    {
        "profile": "BdoubleO100",
        "hermit_key": "7163fbce-39ac-4a02-b836-a991c45d2dd1",
    },
    {
        "profile": "PearlescentMoon",
        "hermit_key": "75c863ae-bb92-486d-911c-53030c552be0",
    },
    {
        "profile": "iskall85",
        "hermit_key": "7ed3587b-e656-4689-90d6-08e11daaf907",
    },
    {
        "profile": "xBCrafted",
        "hermit_key": "826cdcff-ccb0-42c5-9104-fcd4bb4e7f73",
    },
    {
        "profile": "falsesymmetry",
        "hermit_key": "87d91548-6f18-491f-a267-7833caa5d7d8",
    },
    {
        "profile": "cubfan135",
        "hermit_key": "88e2afec-6f2e-4a34-a96a-de61730bd3ca",
    },
    {
        "profile": "Welsknight",
        "hermit_key": "8fc22d29-4bac-4abe-84d4-7920ed4afe47",
    },
    {
        "profile": "Etho",
        "hermit_key": "93b459be-ce4f-4700-b457-c1aa91b3b687",
    },
    {
        "profile": "ZombieCleo",
        "hermit_key": "a3075fa7-ec13-49a2-aa47-6529e8b7daf2",
    },
    {
        "profile": "Mumbo",
        "hermit_key": "ac224782-efff-4296-b08c-dbde8e47abdb",
    },
    {
        "profile": "hypnotizd",
        "hermit_key": "b0015b93-8a5d-461d-9991-3cfa23e3296f",
    },
    {
        "profile": "GoodTimeWithScar",
        "hermit_key": "cae9554c-31be-47e2-ba2b-4b8867adacc5",
    },
    {
        "profile": "Tinfoilchef",
        "hermit_key": "cbf33660-3994-42c3-8d2f-6a1a84d56dea",
    },
    {
        "profile": "Stressmonster101",
        "hermit_key": "cfaefb14-46d5-473b-9e8e-67ecbf119df7",
    },
    {
        "profile": "Keralis1",
        "hermit_key": "ed260cac-54e4-4ee5-b4de-d289f197fa45",
    },
    {
        "profile": "impulseSV",
        "hermit_key": "f6fe2200-609d-4fe6-88b6-529d59ee5b71",
    },
    {
        "profile": "Zedaph",
        "hermit_key": "f9c3c385-f403-403c-b5b7-867e012e9660",
    },
]

local_user = "culley"
# OSX path
path = Path(
    "/Users/{user}/Library/Application Support/minecraft/saves/hermitcraft9/".format(user=local_user)
)
stats_files = list(path.rglob("stats/*.json"))

def minecraft_key(key_name):
    """JSON nodes in minraft files are prefixed with minecraft: - this function removes the prefix so that the keys can be used in plot labels"""
    return "minecraft:{key}".format(key=key_name)


def merge_data_frames(df, df2, df_name):
    """Minecraft stats files contain separate datasets for: mined, crafted, broken, custom, dropped, killed, picked_up, and used. This function merges these datasets based on the keys, creating a column for each statistic"""
    return pd.merge(
        df,
        pd.DataFrame({"minecraft_key": df2.keys(), df_name: df2.values()}),
        how="left",
        left_on="minecraft_key",
        right_on="minecraft_key",
    )

def find_profile_by_filename(path, data_list):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in data_list:
        if dictionary.get('hermit_key') == filename:
            return dictionary.get('profile', 'none')
    
    # Return 'none' if no match found
    return 'none'
    
def world_stats(path):
    hermit = find_profile_by_filename(path, hermits)

    if hermit is None:
        return pd.DataFrame()

    df = pd.read_json(path)
    stats = df["stats"]
    
    broken = stats[minecraft_key("broken")] if minecraft_key("broken") in stats else {}
    crafted = (
        stats[minecraft_key("crafted")] if minecraft_key("crafted") in stats else {}
    )
    custom = stats[minecraft_key("custom")] if minecraft_key("custom") in stats else {}
    dropped = (
        stats[minecraft_key("dropped")] if minecraft_key("dropped") in stats else {}
    )
    killed = stats[minecraft_key("killed")] if minecraft_key("killed") in stats else {}
    mined = stats[minecraft_key("mined")] if minecraft_key("mined") in stats else {}
    picked_up = (
        stats[minecraft_key("picked_up")] if minecraft_key("picked_up") in stats else {}
    )
    used = stats[minecraft_key("used")] if minecraft_key("used") in stats else {}

    df = pd.DataFrame()

    # create unique list of keys from all dictionaries
    df["minecraft_key"] = list(
        set(broken.keys())
        | set(crafted.keys())
        | set(custom.keys())
        | set(dropped.keys())
        | set(killed.keys())
        | set(mined.keys())
        | set(picked_up.keys())
        | set(used.keys())
    )
    df = merge_data_frames(df, broken, "broken")
    df = merge_data_frames(df, crafted, "crafted")
    df = merge_data_frames(df, custom, "custom")
    df = merge_data_frames(df, dropped, "dropped")
    df = merge_data_frames(df, killed, "killed")
    df = merge_data_frames(df, mined, "mined")
    df = merge_data_frames(df, picked_up, "picked_up")
    df = merge_data_frames(df, used, "used")

    

    
    # remove minecraft: prefix
    df["minecraft_key"] = [
        re.sub(r"minecraft:", "", str(x)) for x in df["minecraft_key"]
    ]
    df["hermit"] = hermit
    df["wood_type"] = df["minecraft_key"].str.extract(
        r"(dark_oak|birch|oak|acacia|spruce|jungle|mangrove)"
    )
    df = df.fillna(0)


    df = df.astype(
        {
            "broken": "int",
            "broken": "int",
            "crafted": "int",
            "custom": "int",
            "dropped": "int",
            "killed": "int",
            "mined": "int",
            "picked_up": "int",
            "used": "int",
        }
    )

    return df


minecraft_stats = pd.DataFrame()
parse_errors = pd.DataFrame(columns=["file_name"])
for file_name in stats_files:
   #try:
    minecraft_stats = pd.concat([minecraft_stats, world_stats(file_name)])
    #except:
    #    print(file_name)
    #    parse_errors.loc[len(parse_errors.index)] = file_name.name


# not currently shown on report:
parse_error_summary = parse_errors.groupby("file_name").count()

print(parse_error_summary)

#minecraft_stats = minecraft_stats.set_index(['minecraft_key', 'hermit'])
#if not minecraft_stats.index.is_unique:
#    raise ValueError("The multi-index is not unique.")
    
#minecraft_stats.to_json('hermitcraft9.json')
minecraft_stats.to_csv('hermitcraft9.csv')

#minecraft_stats.head()

#torches = minecraft_stats[minecraft_stats['minecraft_key'] == 'torch']

#torches.head()
#hermits


Empty DataFrame
Columns: []
Index: []


In [142]:
import pandas as pd
import hvplot.pandas
import panel as pn
import base64
import glob

# Load the data
file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

df = df[df["killed"] > 0]

# Sort and define the minecraft_key select widget
sorted_minecraft_keys = sorted(df['minecraft_key'].unique().tolist())
minecraft_key_select = pn.widgets.Select(name='Minecraft Key', options=sorted_minecraft_keys)

# Initial columns to plot
initial_columns = ['broken', 'crafted', 'custom', 'dropped', 'killed', 'mined', 'picked_up', 'used']

# Define the column select widget
column_select = pn.widgets.Select(name='Column', options=initial_columns, value='crafted')

# Placeholder for the plot
plot_placeholder = pn.Column()
hermit_image = pn.Column(align='center')

def find_hermit_key(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('hermit_key', None)
    
    # Return 'none' if no match found
    return None
    
def update_plot(event):
    selected_key = minecraft_key_select.value
    filtered_df = df[df['minecraft_key'] == selected_key]
    
    # Update columns in column_select based on the selected minecraft_key
    columns_with_data = [col for col in initial_columns if filtered_df[col].sum() > 0]
    column_select.options = columns_with_data
    if column_select.value not in columns_with_data:
        column_select.value = columns_with_data[0] if columns_with_data else None

    plot_placeholder.clear()  # Clear existing plot
    
    hermit_image.clear()
    
    if columns_with_data:
        # Create a horizontal bar chart for the selected column
        selected_column = column_select.value
        top_hermits = filtered_df.groupby('hermit')[selected_column].sum()
        # Remove rows where the value is 0
        top_hermits = top_hermits[top_hermits > 0].sort_values(ascending=True)
        top_hermit = top_hermits.idxmax()
        top_hermit_val = top_hermits.max()
        hermit_key = find_hermit_key(top_hermit, hermits)


        # Assuming 'selected_key' and 'selected_column' are defined earlier in your code
        file_path_png = f"blocks/{selected_key}.png"
        file_path_webp = f"mobs/{selected_key}.webp"
        
        # Check if the PNG file exists
        if os.path.exists(file_path_png):
            image_pane = pn.pane.PNG(file_path_png, height=128, width=128)
        elif os.path.exists(file_path_webp):
            with open(file_path_webp, "rb") as image_file:
                encoded_string = base64.b64encode(image_file.read()).decode()
            image_pane = pn.pane.HTML(f'<img src="data:image/webp;base64,{encoded_string}" style="max-height: 128px; max-width: 128px;"/>')
        # If none of the files exist, raise an error
        else:
            image_pane = pn.pane.JPG("mobs/missing.jpg", height=128, width=128)
        
        # Append the image and HTML to hermit_image
        hermit_image.append(pn.Row(image_pane, 
                                   pn.pane.HTML(f'<h2>{selected_key.replace("_", " ").title()}(s) {selected_column.replace("_", " ").title()}</h2>')))
        hermit_image.append(pn.Row(pn.layout.Divider()))
        hermit_image.append(pn.Row(pn.pane.PNG(f"heads/{hermit_key}.png"), width=128))
        hermit_image.append(pn.Row(pn.indicators.Number(name=f"{top_hermit}", value=top_hermit_val, format='{value:,.0f}')))
        

        try:
            top_two = top_hermits.nlargest(2)
            second_hermit_value = top_two.iloc[1]
            second_hermit = top_two.index[1]
            second_hermit_key = find_hermit_key(second_hermit, hermits)
            hermit_image.append(pn.Row(pn.layout.Divider()))
            hermit_image.append(pn.Row(pn.Spacer(height=40)))
            hermit_image.append(pn.Row(pn.pane.PNG(f"heads/{second_hermit_key}.png", width=128)))
            hermit_image.append(pn.Row(pn.indicators.Number(name=f"{second_hermit}", value=second_hermit_value, format='{value:,.0f}')))
        except:
            pass
            
        plot = top_hermits.hvplot.barh(
            title=f'Hermitcraft Season 9 Mob Kill Stats',
            xlabel='Hermit',  
            ylabel=f'{selected_key.replace("_", " ").title()}: {selected_column.replace("_", " ").title()}',
            width=400,
            height=800
        )
        
        def set_toolbar_autohide(plot, element):
            bokeh_plot = plot.state
            bokeh_plot.toolbar.autohide = True
        
        plot.opts(hooks=[set_toolbar_autohide], backend='bokeh')

        plot_placeholder.append(plot)
            
    else:
        plot_placeholder.append(pn.pane.Markdown(f"No data available for {selected_key}."))


update_plot(None)
minecraft_key_select.param.watch(update_plot, 'value')
column_select.param.watch(update_plot, 'value')
layout = pn.Column(pn.Row(pn.Spacer(height=20)), pn.Row(pn.Column(plot_placeholder), hermit_image, pn.Column(minecraft_key_select, column_select)))
pn.serve(layout)


Launching server at http://localhost:60831


In [141]:
import pandas as pd
import panel as pn
import hvplot.pandas

# Load the data
file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

df = df[df["killed"] > 0]

# Sort and define the hermit select widget
sorted_hermits = sorted(df['hermit'].unique().tolist())
hermit_select = pn.widgets.Select(name='Hermit', options=sorted_hermits)

# Placeholder for the plot
plot_placeholder = pn.Column()
hermit_image = pn.Column()

def update_plot(event):
    selected_hermit = hermit_select.value
    filtered_df = df[df['hermit'] == selected_hermit]

    plot_placeholder.clear()  # Clear existing plot
    hermit_image.clear()  # Clear existing hermit_image

    if not filtered_df.empty:
        # Create a vertical bar chart for the top 30 mobs killed by the selected hermit
        top_mobs = filtered_df.groupby('minecraft_key')['killed'].sum().nlargest(30)
        plot = top_mobs.hvplot.bar(
            title=f'{selected_hermit} - Top 30 Mobs Killed',
            xlabel='Mob',  
            ylabel='Kills',
            width=800,
            height=400
        )

        plot_placeholder.append(plot)

        # Loop through the DataFrame rows and append content to hermit_image
        for minecraft_key, kills in top_mobs.items():
            # Assuming 'minecraft_key' corresponds to an image file name
            hermit_image.append(pn.Row(pn.layout.Divider()))
            hermit_image.append(pn.Row(pn.Spacer(height=40)))
            #hermit_image.append(pn.Row(pn.pane.PNG(f"heads/{minecraft_key}.png", width=128)))
            file_path_webp = f"mobs/{minecraft_key}.webp"
            if os.path.exists(file_path_webp):
                with open(file_path_webp, "rb") as image_file:
                    encoded_string = base64.b64encode(image_file.read()).decode()
                image_pane = pn.pane.HTML(f'<img src="data:image/webp;base64,{encoded_string}" style="max-height: 128px; max-width: 128px;"/>')
            else:
                image_pane = pn.pane.JPG("mobs/missing.jpg", height=128, width=128)
            hermit_image.append(image_pane)
            hermit_image.append(pn.Row(pn.indicators.Number(name=f"{minecraft_key}", value=kills, format='{value:,.0f}')))

    else:
        plot_placeholder.append(pn.pane.Markdown(f"No data available for {selected_hermit}."))

update_plot(None)
hermit_select.param.watch(update_plot, 'value')

layout = pn.Column(pn.Row(pn.Spacer(height=20)), pn.Row(plot_placeholder, hermit_image, pn.Column(hermit_select)))
pn.serve(layout)


Launching server at http://localhost:60817


# 100 Top Mob Kill Stats for Hermitcraft Season 9


In [246]:
import re

def add_ordinal_suffix(num):
    # Check for 11-13 because those are the exceptions to the standard rules
    if 11 <= num % 100 <= 13:
        suffix = 'th'
    else:
        # Determine the suffix based on the last digit
        suffixes = {1: 'st', 2: 'nd', 3: 'rd'}
        suffix = suffixes.get(num % 10, 'th')

    return f"{num}{suffix}"
    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide):
    numbers = re.findall(r'\d+', next_slide)
    countdown = 100 - int(numbers[0]) + 2
    if countdown == 1:
        refresh = 10000
    elif countdown < 11:
        refresh = 6000
    elif countdown > 50:
        refresh = 2000
    else:
        refresh = 3000
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Mob Kill Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-5" style="padding-top:130px;">
            <div>
                <h5 class="text-center">Hermitcraft Season 9</h5>
                <h1 class="text-center"><span class="badge text-bg-danger">{countdown}</span></h1>
                <h3 class="text-center">Top Mob Kill Stats</h3>
            </div>
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card no-boarder" style="width: 400px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                                <center>
                                <img src="../{item_image}" style="max-width:128px; max-height:128px;" class="px-2"><br />
                                <span class="badge text-bg-primary">{item_name}</span>
                                </center>
                            </div>
                            <div class="col">
                            <center>
                                <img src="../{hermit_image}" width="128" height="128" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                        </div>
                    </div>
                    <div class="card-body"><center>
                        <h5 class="card-title">{item_name}(s) killed by {hermit_name}</h5>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{amount}</h1></span></p>
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=item_image, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh)


def generate_carousel_item_summary(hermit_name, hermit_image, total_kills, next_slide):
    # Define the HTML structure for the summary slide
    # Replace this with your actual HTML template
    # file:///Users/culley/Documents/clones/HermitcraftStats/slides/slide170.html
    return f"""
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Mob Kill Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, 5000); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>
                <div class="m-5" style="padding-top:130px;">
        <!-- <div><h3 class="text-center">Hermitcraft Season 9 Mob Kill Stats</h3></div> -->
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card" style="width: 400px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                                <img src="../{hermit_image}" width="128" height="128" class="px-2">
                                <span class="badge text-bg-success">{hermit_name}</span>
                            </div>
                        </div>
                    </div>
                    <div class="card-body">
                        <hr>
                        <h3 class="card-title"><span class="badge text-bg-danger">{hermit_name}'s total kills</span></h3>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{total_kills}</h1></span></p>
                    </div>
                </div>
            </div>
        </div>
        
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>
    </body>
    </html>
    """

file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

df = df[df["killed"] > 0]
df = df[df["hermit"] != 'none']

grouped_df = df.groupby(['hermit', 'minecraft_key'])['killed'].sum().reset_index()
# Track the current hermit and their total kills
current_hermit = None
total_kills = 0
slide_index = 1
show_summary_slides = False

# Hermit summary data
#sorted_df = grouped_df.sort_values(by=['hermit', 'minecraft_key'])
# top killed data
sorted_df = grouped_df.sort_values(by=['killed'], ascending=[False])
sorted_df = sorted_df.head(100).sort_values(by=['killed'], ascending=[True])

#sorted_df = sorted_df[sorted_df["killed"] > 500]


def find_hermit_key(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('hermit_key', None)
    
    # Return 'none' if no match found
    return None

# Ensure the 'slides' subdirectory exists
os.makedirs('slides', exist_ok=True)

for index, row in sorted_df.iterrows():
    if current_hermit != row['hermit']:
        if current_hermit is not None and show_summary_slides is True:
            # Add a summary slide for the previous hermit
            hermit_key = find_hermit_key(current_hermit, hermits)
            summary_html = generate_carousel_item_summary(current_hermit, f"heads/{hermit_key}.png", "{:,}".format(total_kills), f"slide{slide_index + 1}.html")
            with open(f"slides/slide{slide_index}.html", "w") as file:
                file.write(summary_html)
            slide_index += 1
        current_hermit = row['hermit']
        total_kills = 0

    total_kills += row['killed']
    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_key = find_hermit_key(row['hermit'], hermits)
    item_html = generate_carousel_item(
        f"heads/{hermit_key}.png",
        f"mobs/{row['minecraft_key']}.webp",
        row['hermit'],
        row['minecraft_key'].replace("_", " ").title(),
        "{:,}".format(row['killed']),
        next_slide
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1
    
# Add a summary slide for the last hermit
if current_hermit is not None and show_summary_slides is True:
    hermit_key = find_hermit_key(current_hermit, hermits)
    summary_html = generate_carousel_item_summary(current_hermit, f"heads/{hermit_key}.png", total_kills, "slide1.html")
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(summary_html)


    
#print(df["hermit"].unique())
